# STIPS Advanced II – PSFs, Adding Sources

This portion of the Advanced tutorial concerns the generation and manipulation of PSFs using STIPS functions.  As with Advanced I, this notebook assumes both that you already have STIPS installed (see [Installing STIPS](https://stsci-stips.readthedocs.io/en/latest/installation.html) if not) and that you are comfortable with basic usage of STIPS functionalities (see [STIPS Basic Tutorial](https://stips.readthedocs.io/en/latest/basic_tutorial.html), or the Basic Tutorial notebook, if not).

#### Note:
The ```makePSF``` package is still undergoing development –– pending the inclusion of logging functions and error catching, as well as some functional optimization.  This tutorial will be updated to reflect this in future releases.

## Checking STIPS Import, Adding Modules

Before beginning, check again that the STIPS import is correct, and import and configure a few modules we'll be needing.  Additionally, ensure that the generated files from the Basic Tutorial are visible in your "notebooks" folder/the directory from which you are working.

In [ ]:
from astropy.io import fits
from glob import glob
from matplotlib import style
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import stips

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

matplotlib.rcParams['axes.grid'] = False
matplotlib.rcParams['image.origin'] = 'lower'

print(stips.__env__report__)

In [ ]:
file_list = glob('notebooks_data/notebook_example*')
print(file_list)

## Generating and Manipulating a PSF

PSFs can be generated using FITS input files and the makePSF module within STIPS.  For this example, we will be using the example PSF file "psf_WFI_2.0.0_F129_sca01.fits", a PSF generated with Roman WFI F129.  This file is available in the "notebooks" directory of STIPS.  The generated PSF will be returned as an array.

In [ ]:
example_file = fits.open('notebooks_data/psf_WFI_2.0.0_F129_sca01.fits')

test_psf = stips.utilities.makePSF.make_epsf(example_file[0].data[0])

print(test_psf)

We can view a plot of our generated PSF using pyplot.

In [ ]:
fig1 = plt.figure()
im1 = plt.matshow(test_psf)
plt.savefig('notebooks_data/adv_notebook_epsf_orig.jpeg')

The PSF can be manipulated in a variety of ways –– for purposes of this tutorial, we will perform a bicubic interpolation on our PSF, as well as calculate the fraction of light that should fall on a given pixel.  This will employ two of makePSFs primary functions, ```makePSF.bicubic``` and ```makePSF.real_psf```.

### Bicubic Interpolation

For our bicubic interpolation, we will use the following input parameters, selecting a reference point close to the source in the hopes of seeing more engaging results:

* Reference x pixel (```ix```) – 85
* Reference y pixel (```iy```) – 85
* Pixel phase along the x-direction (```fx```) – 5
* Pixel phase along the y-direction (```fy```) – 5

In [ ]:
ix = iy = 85
fx = fy = 5

stips.utilities.makePSF.bicubic(test_psf, iy, ix, fx, fy)

Noted in the image below is the location at which the interpolation was performed (cropped for visibility).

In [ ]:
def plot_square_cutout(data, x_mid, y_mid, half):
    '''
    x_mid: index of middle x pixel in desired cutout region
    y_mid: index of middle y pixel in desired cutout region
    half: distance in pixels from middle pixel to edge of cutout
    '''
    
    # create plot
    fig, ax = plt.subplots(figsize=(4,4))
    ax.imshow(data[x_mid - half:x_mid + half,
                   y_mid - half:y_mid + half])
    
    # create custom formatters for cutout region axes (run once)
    ticks_x = ticker.FuncFormatter(lambda x, pos: f"{x + x_mid - half:.0f}")
    ticks_y = ticker.FuncFormatter(lambda y, pos: f"{y + y_mid - half:.0f}")
    
    # replace default tick labels with cutout pixel indices
    ax.set_xticks([x for x in ax.get_xticks() if 0 <= x < half*2])
    ax.set_yticks([y for y in ax.get_xticks() if 0 <= y < half*2])
    ax.xaxis.set_major_formatter(ticks_x)
    ax.yaxis.set_major_formatter(ticks_y)
    
    return fig, ax

In [ ]:
# Plot parameters
x_mid = 88
y_mid = 88
half = 30

fig0, ax0 = plot_square_cutout(test_psf, x_mid, y_mid, half)

circ_bicubic = plt.Circle((half - 3, half - 3), 3, color='r', fill=False)

ax0.add_patch(circ_bicubic)

### Fractional Light

Using ```makePSF.real_psf```, we can caculate the fraction of light from our epsf that should fall on any given pixel.  We will use the following parameters:

* Relative location of source along x-axis (```dx```) (note that 0 the default the center of the boxsize) –– 5
* Relative location of the source along y-axis (```dy```) –– 5
* Center of the input psf model (```psf_center```) (our PSF has dimensions of 177 by 177 px) –– 88
* PSF boxsize (```boxsize```) –– 177

#### Note:

Currently, the default value of ```psf_center``` is set to 177, and default boxsize 44 –– this will be adjusted in future updates.

In [ ]:
dx = dy = 5

#The code below is designed to modify the default values for the center
#and size of the PSF to fit the PSF being used –– this will be addressed
#in future releases.

psf_middle = round((test_psf[0].shape[0]-1)/2)

PSF_BOXSIZE = np.floor(psf_middle)/4

stips.utilities.makePSF.real_psf(dx, dy, test_psf, boxsize=PSF_BOXSIZE, psf_center=psf_middle)

Noted below is the location at which fractional light was evaluated.

In [ ]:
fig1_2, ax1_2 = plot_square_cutout(test_psf, x_mid, y_mid, half)

circ_real_psf = plt.Circle((half + 5, half + 5), 3, color='r', fill=False)
ax1_2.add_patch(circ_real_psf)

## Adding New Sources to a Scene

To add a new source to our generated scene from the Basic Tutorial, we will use the ```place_source``` function from ```makePSF``` and insert a source with the following characteristics:

* Location (px)
    * x = 2000
    * y = 2000
* Flux = 15

Note that, by default, the PSF upscale is 4.

#### Note:

Currently, the default value of ```psf_center``` is set to 177, and default boxsize 44 –– this will be adjusted in future updates.

### Original Scene

Re-open the original scene from the saved output file –– by default, this file is "notebook_example_1_0.fits".  We will limit our area of view to the range in which the added source will be visible, between 1950 and 2150 pixels on each axis.

In [ ]:
with fits.open('notebooks_data/notebook_example_1_0.fits') as result_file:
    result_data = result_file[1].data

# Plot parameters
x_mid2 = 2000
y_mid2 = 2000
half2 = 50

fig2, ax2 = plot_square_cutout(result_data, x_mid2, y_mid2, half2)

circ_no_source = plt.Circle((half2, half2), 3, color='r', fill=False)
ax2.add_patch(circ_no_source)

### Add New Source

We now use the ```makePSF.place_source``` function to add in our test PSF as a source (noting that the ```boxsize``` and ```psf_center``` parameters must be modified from their defaults for this particular PSF, as was the case with the ```makePSF.real_psf example```).

In [ ]:
#The code below is designed to modify the default values for the center
#and size of the PSF to fit the PSF being used –– this will be addressed
#in future releases.

psf_middle = round((test_psf[0].shape[0]-1)/2)

PSF_BOXSIZE = np.floor(psf_middle)/4

added_source = stips.utilities.makePSF.place_source(2000, 2000, 3000, result_data, test_psf, boxsize=PSF_BOXSIZE, psf_center=psf_middle)

We can now open our modified image and view our added source.

In [ ]:
fig2_1, ax2_1 = plot_square_cutout(result_data, x_mid2, y_mid2, half2)

circ_source = plt.Circle((half2, half2), 3, color='r', fill=False)
ax2_1.add_patch(circ_source)

## Conclusion

This concludes the second portion of the advanced tutorial.  If you have further questions, check out the [STIPS documentation](https://stips.readthedocs.io/en/latest/), or reach out to the STIPS Helpdesk via email at help@stsci.edu with the subject line "STIPS Question".